In [1]:
import os
import fiona
import geopandas as gpd
from shapely.geometry import shape
import matplotlib.pyplot as plt
from shapely.ops import unary_union

rivers = {}

output_directory = '../data/external/shp/clipped_rivers_grid_cells_ogr'

for i,riv in enumerate(os.listdir(output_directory)):
    # Check if the file is a shapefile
    try:
        if riv.endswith('.shp'):
            print(f"Processing file: {riv}")
            river = gpd.read_file(os.path.join(output_directory, riv))
            combined_river = unary_union(river.geometry)
            
            total_length = combined_river.length  # Longitud total en metros

            # Calcular el área del bounding box de la geometría combinada
            bounding_box = river.total_bounds  # [minx, miny, maxx, maxy]
            width = bounding_box[2] - bounding_box[0]
            height = bounding_box[3] - bounding_box[1]
            area = width * height  # Área en metros 

            bounding_box_diagonal = ((bounding_box[2] - bounding_box[0])**2 + (bounding_box[3] - bounding_box[1])**2)**0.5

            length_percentage = (total_length / bounding_box_diagonal) * 100  # Comparación en función de la diagonal
            
            # Definir umbrales
            min_area_threshold = 225_000  # Área mínima en m² (0.225 km²)
            min_length_threshold = 300  # Longitud mínima en metros

            # Plot the GeoDataFrame
            river = river.to_crs("EPSG:4326")

            if area > min_area_threshold and total_length > min_length_threshold and length_percentage > 50:
                river = river.to_crs("EPSG:4326")
                rivers[f'cell_{i}'] = river

    except Exception as e:
        print(f"Error processing file: {riv}",e)
        pass
        

Processing file: rivers_grid_cell_666.shp
Processing file: rivers_grid_cell_320.shp
Processing file: rivers_grid_cell_187.shp
Processing file: rivers_grid_cell_754.shp
Processing file: rivers_grid_cell_567.shp
Processing file: rivers_grid_cell_490.shp
Processing file: rivers_grid_cell_328.shp
Processing file: rivers_grid_cell_597.shp
Processing file: rivers_grid_cell_242.shp
Processing file: rivers_grid_cell_304.shp
Processing file: rivers_grid_cell_109.shp
Processing file: rivers_grid_cell_285.shp
Processing file: rivers_grid_cell_405.shp
Processing file: rivers_grid_cell_199.shp
Processing file: rivers_grid_cell_356.shp
Processing file: rivers_grid_cell_418.shp
Processing file: rivers_grid_cell_303.shp
Processing file: rivers_grid_cell_478.shp
Processing file: rivers_grid_cell_212.shp
Processing file: rivers_grid_cell_444.shp
Processing file: rivers_grid_cell_416.shp
Processing file: rivers_grid_cell_442.shp
Processing file: rivers_grid_cell_246.shp
Processing file: rivers_grid_cell_

In [16]:
import ee
import datetime
# Inicializar la API de Earth Engine
ee.Initialize()

def maskS2clouds(image):
    qa = image.select('QA_PIXEL')

    dilatedCloudsBitMask = 1 << 1  # Mask area around clouds
    cirrusBitMask = 1 << 2  # Mask cirrus clouds
    cloudsBitMask = 1 << 3  # Mask clouds
    cloudShadowBitMask = 1 << 4  # Mask cloud shadows

    # Both flags should be set to zero, indicating clear conditions.
    mask = (qa.bitwiseAnd(dilatedCloudsBitMask).eq(0)
            .And(qa.bitwiseAnd(cirrusBitMask).eq(0))
            .And(qa.bitwiseAnd(cloudsBitMask).eq(0))
            .And(qa.bitwiseAnd(cloudShadowBitMask).eq(0)))

    return image.updateMask(mask)

def addNDVI(image):
    ndvi = (image.multiply(2.75e-05)
            .subtract(0.2)
            .normalizedDifference(['SR_B5', 'SR_B4'])
            .rename('ndvi'))
    return image.addBands(ndvi)

def get_active_task_count():
    tasks = ee.batch.Task.list()  # Lista todas las tareas
    active_tasks = [task for task in tasks if task.state in ['RUNNING', 'READY']]
    return len(active_tasks)

def export_landsat_ndvi_by_month(zone_name, start_date, end_date, coordinates):
    # Definir la geometría del rectángulo con las coordenadas proporcionadas
    geometry = ee.Geometry.Rectangle(coordinates)

    # Convertir las fechas de entrada a objetos datetime
    start_date = datetime.datetime.strptime(start_date, '%Y-%m-%d')
    end_date = datetime.datetime.strptime(end_date, '%Y-%m-%d')

    # Iterar por cada mes entre las fechas de inicio y fin
    current_date = start_date
    while current_date < end_date:
        # Calcular la fecha de fin del mes actual
        next_month = current_date.replace(day=28) + datetime.timedelta(days=4)  # Ir al próximo mes
        next_month = next_month.replace(day=1)  # Establecer el primer día del próximo mes

        # Filtrar el conjunto de datos por el mes actual
        dataset = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
                    .filterDate(current_date.strftime('%Y-%m-%d'), next_month.strftime('%Y-%m-%d')) \
                    .map(maskS2clouds) \
                    .map(addNDVI)

        # Seleccionar y calcular la media del NDVI para el mes
        image = dataset.select('ndvi').mean().clip(geometry)

        # Exportar la imagen a Google Drive para el mes
        task = ee.batch.Export.image.toDrive(
            image=image,
            description=f'landsat_{zone_name}_{current_date.strftime("%Y_%m")}_ndvi',
            scale=30,
            region=geometry,
            fileNamePrefix=f'{zone_name}_{current_date.strftime("%Y_%m")}_ndvi',
            folder='landsat_ndvi'
        )
        task.start()

        print(f"Export task for {zone_name} - {current_date.strftime('%Y-%m')} started.")
        
        while get_active_task_count() >= 3000:
            print("Task limit reached. Waiting for some tasks to complete...")
            time.sleep(60)  # Esperar 1 minuto antes de verificar nuevamente


        # Pasar al siguiente mes
        current_date = next_month


In [5]:
start_date = '2013-03-01'
end_date = '2019-12-31'
for num, (k,v) in enumerate(rivers.items()):
    print(num)
    coordinates = list(v.total_bounds)
    zone_name = k
    export_landsat_ndvi_by_month(zone_name, start_date, end_date, coordinates)
 

0
Export task for cell_4 - 2013-03 started.
Export task for cell_4 - 2013-04 started.
Export task for cell_4 - 2013-05 started.
Export task for cell_4 - 2013-06 started.
Export task for cell_4 - 2013-07 started.
Export task for cell_4 - 2013-08 started.
Export task for cell_4 - 2013-09 started.
Export task for cell_4 - 2013-10 started.
Export task for cell_4 - 2013-11 started.
Export task for cell_4 - 2013-12 started.
Export task for cell_4 - 2014-01 started.
Export task for cell_4 - 2014-02 started.
Export task for cell_4 - 2014-03 started.
Export task for cell_4 - 2014-04 started.
Export task for cell_4 - 2014-05 started.
Export task for cell_4 - 2014-06 started.
Export task for cell_4 - 2014-07 started.
Export task for cell_4 - 2014-08 started.
Export task for cell_4 - 2014-09 started.
Export task for cell_4 - 2014-10 started.
Export task for cell_4 - 2014-11 started.
Export task for cell_4 - 2014-12 started.
Export task for cell_4 - 2015-01 started.
Export task for cell_4 - 2015-02

EEException: Too many tasks already in the queue (3000, limit 3000).

In [10]:
from itertools import islice
rivers_left = dict(islice(rivers.items(), 35, None))
len(rivers_left)

451

In [ ]:
start_date = '2013-03-01'
end_date = '2019-12-31'
for num, (k,v) in enumerate(rivers_left.items()):
    print(num)
    coordinates = list(v.total_bounds)
    zone_name = k
    export_landsat_ndvi_by_month(zone_name, start_date, end_date, coordinates)
 

0
Export task for cell_163 - 2013-03 started.
Export task for cell_163 - 2013-04 started.
Export task for cell_163 - 2013-05 started.
Export task for cell_163 - 2013-06 started.
Export task for cell_163 - 2013-07 started.
Export task for cell_163 - 2013-08 started.
Export task for cell_163 - 2013-09 started.
Export task for cell_163 - 2013-10 started.
Export task for cell_163 - 2013-11 started.
Export task for cell_163 - 2013-12 started.
Export task for cell_163 - 2014-01 started.
Export task for cell_163 - 2014-02 started.
Export task for cell_163 - 2014-03 started.
Export task for cell_163 - 2014-04 started.
Export task for cell_163 - 2014-05 started.
Export task for cell_163 - 2014-06 started.
Export task for cell_163 - 2014-07 started.
Export task for cell_163 - 2014-08 started.
Export task for cell_163 - 2014-09 started.
Export task for cell_163 - 2014-10 started.
Export task for cell_163 - 2014-11 started.
Export task for cell_163 - 2014-12 started.
Export task for cell_163 - 201